### WesterLab scRNA_seq analysis

Tools for downstream analysis of scRNA-seq differential expression data

Subroutines created by Anthony Moussa and presented in Moussa and Wester, 2022 https://doi.org/10.1101/2022.06.14.496156

### Importing libraries

In [7]:
import numpy as np
import pandas as pd

### Function Name

description: parameters and result


body of text for code with comments for key algorithms as would be described in pseudocode

### GOPrep
pass a dataframe with compiled DE data to be labeled and filtered for GO terms

df = combined_df

FinalGO = last GO term in combined_df

In [8]:
def GOPrep(df, FinalGO):
    
    # label GO relevant genes
    df.loc[df['Gene'].str.lower().isin(df["CCA.1"].str.lower()), 'CCA'] = 1

    df.loc[df['Gene'].str.lower().isin(df["RCCA.1"].str.lower()), 'RCCA'] = 1

    df.loc[df['Gene'].str.lower().isin(df["RTSS.1"].str.lower()), 'RTSS'] = 1

    # select GO relevant genes
    while True:
        filtered = input("Labeled! Would you like to filter out non-GO genes and non-DE genes (y/n):")
        if filtered == 'y':
            df = df.loc[(df['CCA'] == 1) | (df['RCCA'] == 1) | (df['RTSS'] == 1)]
            df = df.loc[~(df['p_val_adj'] > .05)]
            break
        elif filtered == 'n':
            break
        else:
            print("Please enter valid input")
    
    GOPrep = df
    
    cutoff = GOPrep.columns.get_loc(FinalGO)+1
   
    # drop remnant columns from filtering
    GOPrep.drop(GOPrep.columns[cutoff:], axis=1, inplace=True)
    return GOPrep

### Subsetter
pass a dataframe to be subsetted and sorted

df = combined_df

In [9]:
def subsetter(df):

    total_df = df
    
    #subset for GO terms
    while True:
        GO = input("Subset for all GOs (all), Cell-Cell Adhesion (CCA), Regulation of CCA (RCCA), Regulation of Trans-Synaptic signaling (RTSS): ")
        if GO == "all":
            break
        elif GO == "CCA" or GO == "RCCA" or GO == "RTSS":
            total_df = total_df.loc[(total_df[GO] == 1)]
            break
        else:
            print("Please enter valid input")

    #subset for cell classes
    while True:
        cellclass = input("Subset for all data (all), glutamatergic (Glut) or GABAergic (GABA): ")
        if cellclass == "all":
            break
        elif cellclass == "Glut" or cellclass == "GABA":
            total_df = total_df.loc[total_df["Class"] == cellclass]
            break
        else:
            print("Please enter valid input")

    #subset for brain regions
    while True:
        brain_region = input("Subset for all data (all), ALM vs VISp, ALM, VISp, or combination (combo): ")
        if brain_region == "all":
            break
        elif brain_region == "ALM vs VISp" or brain_region == "ALM" or brain_region == "VISp":
            total_df = total_df.loc[total_df["brain_region"] == brain_region]
            break
        elif brain_region == "combo":
            while True:
                Regions = input(f"Available brain_regions: {list(df['brain_region'].unique())} \n"
                               "Please (re)enter list of brain_regions from those available (comma separated values, no spaces after commas):  ")
                more = "n"
                for x in Regions.split(","):
                    if not (x in list(df['brain_region'].unique())):
                                 print(f"{x} is not an available brain region")
                                 more = "y"
                                 L=[]

                if more == "n":
                    more = input(f"Content with current brain regions:{Regions.split(',')} \n" 
                                 "(y/n) (n restarts function): ")
                    if more != "n":
                        total_df = total_df.loc[df['brain_region'].isin(Regions.split(','))]
                        break
            break
        else:
            print("Please enter valid input")
    
    #subset for subclasses
    if input("Subset for specific subclasses (y/n) (y = specifics; n = all subclasses): ") == "y":
        while True:
            SubClasses = input(f"Available subclasses: {list(total_df['Subclass'].unique())} \n"
                           "Please (re)enter list of subclasses from those available (comma separated values, no spaces after commas):  ")
            more = "n"
            for x in SubClasses.split(","):
                if not (x in list(total_df['Subclass'].unique())):
                             print(f"{x} is not an available subclass")
                             more = "y"

            if more == "n":
                more = input(f"Content with current subclasses:{SubClasses.split(',')} \n" 
                             "(y/n) (n restarts function): ")
                if more != "n":
                    total_df = total_df.loc[total_df['Subclass'].isin(SubClasses.split(","))]
                    break
    
    #sorter for plotting
    if input("Would you like to sort subsetted data? (y/n) ") == "y":
        while True:        
            Sort = input("Sort by GABAergic/Glutmatergic (C), brain region (R), or subclass (SC) (list order, comma separated): ")

            D = {"C": "Class", "R": "brain_region", "SC": "subclass"}

            more = "n"
            L = Sort.split(",")
            for x in L:
                if not (x in ["C","R","SC"]):
                             print(f"{x} is not an available sort parameter")
                             more = "y"

            if more == "n":
                more = input(f"Content with current subclasses:{Sort.split(',')} \n" 
                             "(y/n) (n restarts function): ")

                if more != "n":
                    total_df = total_df.sort_values([D[L[0]],D[L[1]], D[L[2]]])
                    break     
                                                       
    return GO, cellclass, brain_region, total_df

### clinExploder
pass a clinical df (e.g. uploaded SFARI ASD list) with one or more columns with csv strings and will use Pandas Explode function to split/explode such columns into separate rows.

e.g break up row with multiple clinical descriptions into multiple rows

preset for "conditions" column

df_clin - used passed clinical df. e.g. SFARI list with columns for genes and ASD classification

In [1]:
def clinExploder(df_clin):
    
    while True:
        SFARI = input("Are you passing the ASD SFARI gene list with 'Condition' and 'syndromic' columns? (y/n):" )
        if SFARI == "y":
            df_clin.loc[df_clin['Condition'] == 3.0, 'Condition'] = "Suggestive Evidence"
            df_clin.loc[df_clin['Condition'] == 2.0, 'Condition'] = "Strong Candidate"
            df_clin.loc[df_clin['Condition'] == 1.0, 'Condition'] = "High Confidence"
            df_clin.loc[df_clin['Condition'] == np.nan, 'Condition'] = ""
            df_clin.loc[df_clin['syndromic'] == 1.0, 'syndromic'] = "Syndromic"
            df_clin.loc[df_clin['syndromic'] == 0, 'syndromic'] = ""
            df_clin.loc[df_clin['Condition'].isnull(), 'Condition'] = ""
            df_clin.loc[(df_clin['Condition'] != "") & (df_clin['syndromic'] == 'Syndromic'), 'Conditions'] = df_clin["Condition"] + "," + df_clin["syndromic"]
            df_clin.loc[(df_clin['Condition'] == "") & (df_clin['syndromic'] == 'Syndromic'), 'Conditions'] = df_clin["syndromic"]
            df_clin.loc[(df_clin['syndromic'] == ""), 'Conditions'] = df_clin["Condition"]
            break
        elif SFARI == 'n':
            break
        else:
            print("Please enter valid input")
       
    df_clin = df_clin[['Gene','Conditions']]

    df_clin['Conditions'] = df_clin['Conditions'].apply(lambda x: x.split(','))

    df_clin = df_clin.explode('Conditions')

    df_clin.columns = ['Gene','Condition']

    df_clin = df_clin.reset_index(drop =True)
    
    return df_clin

### ClinPrep

prepares a filtered clinical df for all downstream clinical functions

df_clin = df that captures all potential clinical categories (can adjust based on stringency of clinical classfication); columns required: Gene, Condition

df_DEGenes = df with all possible DE Genes; columns required: DEGlut, DEGABA, DE

In [10]:
def ClinPrep(df_clin,df_DEGenes):
      
    # Break up rows with comma delimits
    df_clin = pd.concat([pd.Series(row['Condition'], row['Gene'].split(', '))              
                        for _, row in df_clin.iterrows()]).reset_index()

    df_clin.columns = ['Gene', 'Conditions']

    df_clin = df_clin.drop_duplicates(subset=['Gene', 'Conditions'], keep ='first')

    # Filter for clinically relevant DE genes; most important part of function
    df_clin = df_clin.loc[df_clin["Gene"].str.lower().isin(df_DEGenes["DE"].str.lower())]

    # label for gene origin; gives high level overview of Glutamatergic/GABAergic clinical contributions
    df_clin["Class"] = 0
    df_clin.loc[df_clin["Gene"].str.lower().isin(df_DEGenes["DE Glut"].str.lower()), "Class"] = 1
    df_clin.loc[df_clin["Gene"].str.lower().isin(df_DEGenes["DE GABA"].str.lower()), "Class"] = df_clin["Class"]-1
    df_ClinPrep = df_clin
    
    return df_ClinPrep

### ClinicalFilter

"select clinically relevant genes of interest" from ClinPrep output - i.e. subsets ClinPrep for only genes relevant to conditions in narrowed list; will remove condition labels for gene that are clinically relevant but not in provided dictionary

df = output of ClinPrep

df_ClinList = user passed clinical df that is converted into dictionary and informs which conditions to keep and how they are labeled **relies on preset ConditionList that is manually made

In [11]:
def ClinicalFilter (df,df_ClinList):
    
    clinical = df_ClinList

    d = clinical.groupby('Narrow')['Conditions'].apply(list).to_dict()
    
    df['Found'] = 0
    
#   annotates genes found in dictionary: key = user meta condition (Neurodevelopmental), value = database condition name
    for key, value in d.items():
        df.loc[df["Conditions"].isin(value), ["Conditions", 'Found']] = [key,1]

#   offers option to remove genes not found in dictionary
    while True:
        remove = input("Would you like to remove genes not found associated with a condition in your condition dictionary but still clinically relevant? (y/n):")
        if remove == 'y':
            for key, value in d.items():
                df.loc[df['Found'] != 1, ['Gene',"Conditions"]] = ['NAN','NAN']
            break
        elif remove == 'n':
            break
        else:
            print("Please enter valid input")
    
    df = df.drop_duplicates(subset=['Gene', 'Conditions'], keep ='first')

    return df

### ClinLabel

"label DE genes by selected clinically relevant genes" from output of ClinPrep

takes a df_clinical and df_DE and filters then labels DE genes which are clinically relevant

df_clinical = output from ClinPrep

df_DE = user passed df of all DE genes

df_ClinList = user passed df of all conditions of interest for analysis

In [12]:
#labels gene by conditions (comma separated strings)

def ClinLabel(df_clinical,df_DE,df_ClinList):
    
    df_clinical = ClinicalFilter(df_clinical,df_ClinList)

    #Offers option to annotate all of dataset or just clinically relevant DE genes
    while True: 
        remove = input("Would you like to remove non-clinically relevant DE genes (y/n): ")
        if (remove == 'y'):
            df_DE = df_DE.loc[df_DE["Gene"].str.lower().isin(df_clinical["Gene"].str.lower())]
            df_DE = df_DE.reset_index(drop=True)
            break
        elif remove == 'n':
            break
        else:
            print("Please enter valid input")

    for gene in df_DE["Gene"]:
        clinical_gene = df_clinical.loc[df_clinical['Gene'].str.lower() == gene.lower()]
        #solves issue of ASD/Neurodev vs Neurodev/ASD
        cList = sorted(clinical_gene['Conditions'].tolist())
        my_string = ','.join(cList)
        df_DE.loc[df_DE['Gene'] == gene, "Conditions"] = my_string

    return df_DE

### DECollapser
takes a filtered and labeled df and gives DE genes, noting subclass origin (ALM-specific, VISp-specific, conserved, unique)

Collapser designed for level 3 granularity (brain region and subclass 1 vs 1)


In [14]:
def DECollapser(df):
    
    result = subsetter(df)
    GO = result[0]
    cellclass = result[1]
    brain_region = result[2] 
    df = result[3]

    comparisons = df['Comparison'].unique()

    while True:
        clin = input('Are you passing in a df labeled with conditions? (y/n): ')
        if clin == "y":
            break
        elif clin == "n":
            break
        else:
            print("Please enter valid input")

    rows = []
    
#     create empty df to store DE test data that is different between brain regions
    df_unconserved = df.loc[df['Class'] == 0]
    
    for comparison1 in comparisons:
            df1 = df.loc[df["Comparison"] == comparison1]
            df1 = df1.reset_index(drop=True)
            for comparison2 in comparisons:
                    df2 = df.loc[df["Comparison"] == comparison2]
                    df2 = df2.reset_index(drop=True)
                    
        #             indicates we are doing a same subclass comparison
                    if (df1['Comparison'][0] != df2['Comparison'][0]) & (df1['Subclass'][0] == df2['Subclass'][0]):
                        df_all = df.loc[(df["Comparison"] == comparison1) | (df["Comparison"] == comparison2)]
                        
#                       default for merge is inner: use intersection of keys from both frames - find genes present in BOTH df, hence intersection
                        df_intersection = df1.merge(df2.drop_duplicates(), on=['Gene'],
                                    indicator=True)
                        df_intersection = df_intersection.reset_index(drop=True)

                        for gene in list(df_all['Gene']):
                            #flags for all below
                            brain_region = 2
                            expression = -1000
                            direction = "specific to subclass"
                            regulation = "NAN"
                            difference = "specific to subclass"
                            ALM = False
                            VISp = False
                            CCA = float(list(df_all.loc[df_all['Gene'] == gene]['CCA'])[0])
                            RCCA = float(list(df_all.loc[df_all['Gene'] == gene]['RCCA'])[0])
                            RTSS = float(list(df_all.loc[df_all['Gene'] == gene]['RTSS'])[0])
                            neuronalclass = f"{str(df_intersection['Class_x'][0])}"
                            subclass = f"{str(df_intersection['Subclass_x'][0])}"
                            conditions = "Conditions Unlabeled"

                            if gene in list(df_intersection['Gene']):
                                brain_region = 'ALM and VISp'
                                
#                             gene has to be in ALM and VISp since it is in intersection (here we find brain region origin of gene in DE test)
                                if(df1['brain_region'][0] == 'ALM'):
                                    ALMFC = float(df1.loc[df1["Gene"] == gene]['avg_logFC'])
                                elif(df1['brain_region'][0] == 'VISp'):
                                    VISpFC = float(df1.loc[df1["Gene"] == gene]['avg_logFC'])
                                    
                                if(df2['brain_region'][0] == 'ALM'):
                                    ALMFC = float(df2.loc[df2["Gene"] == gene]['avg_logFC'])
                                elif(df2['brain_region'][0] == 'VISp'):
                                    VISpFC = float(df2.loc[df2["Gene"] == gene]['avg_logFC'])
                                
                                expression = (ALMFC + VISpFC)/2
                                difference = ALMFC/VISpFC
                                intersection = True
                                
                                if clin == 'y':
                                    conditions = f"{str(list(df_intersection.loc[df_intersection['Gene'] == gene]['Conditions_x'])[0])}"

                                if ((ALMFC <0) and (VISpFC <0)) or ((ALMFC >0) and (VISpFC >0)):
                                    direction = "same"
                                    regulation = signer(expression)

#                               for genes that are DE in opposite directions (unconserved) between brain regions, collect in new df
                                else:
                                    diff1 = df1.loc[df1['Gene'] == gene]
                                    diff2 = df2.loc[df2['Gene'] == gene]
                
                                    df_unconservedprep = pd.concat([diff1,diff2])
                                    df_unconserved = pd.concat([df_unconserved,df_unconservedprep],ignore_index = True)
                        
                                    direction = "different"
                                    regulation = "differs between subclasses"

                            # gene wasn't DE in both ALM and VISp; let's find which brain region it was DE in

                            elif ((gene in list(df1['Gene'])) & (df1['brain_region'][0] == 'ALM')):
                                g1FC = float(df1.loc[df1["Gene"] == gene]['avg_logFC'])
                                expression = g1FC
                                brain_region = 'ALM'
                                regulation = signer(expression)
                                
                                if clin == 'y':
                                    conditions = f"{str(list(df1.loc[df1['Gene'] == gene]['Conditions'])[0])}"

                            elif ((gene in list(df1['Gene'])) & (df1['brain_region'][0] == 'VISp')):
                                g1FC = float(df1.loc[df1["Gene"] == gene]['avg_logFC'])
                                expression = g1FC
                                brain_region = 'VISp'
                                regulation = signer(expression)

                                if clin == 'y':                                
                                    conditions = f"{str(list(df1.loc[df1['Gene'] == gene]['Conditions'])[0])}"


                            elif ((gene in list(df2['Gene'])) & (df2['brain_region'][0] == 'VISp')):
                                g2FC = float(df2.loc[df2["Gene"] == gene]['avg_logFC'])
                                expression = g2FC
                                brain_region = 'VISp'
                                regulation = signer(expression)
                                if clin == 'y':
                                    conditions = f"{str(list(df2.loc[df2['Gene'] == gene]['Conditions'])[0])}"


                            elif ((gene in list(df2['Gene'])) & (df2['brain_region'][0] == 'ALM')):
                                g2FC = float(df2.loc[df2["Gene"] == gene]['avg_logFC'])
                                expression = g2FC
                                brain_region = 'ALM'
                                regulation = signer(expression)
                                if clin == 'y':
                                    conditions = f"{str(list(df2.loc[df2['Gene'] == gene]['Conditions'])[0])}"
                                
                            if (brain_region == 'ALM'):
                                rows.append([neuronalclass, subclass, gene, brain_region, conditions, CCA, RCCA, RTSS, expression, expression, 0, direction, regulation, difference])
                            elif (brain_region == 'VISp'):
                                rows.append([neuronalclass, subclass, gene, brain_region, conditions, CCA, RCCA, RTSS, expression, 0, expression, direction, regulation, difference])
                            else:
                                rows.append([neuronalclass, subclass, gene,  brain_region, conditions, CCA, RCCA, RTSS, expression, ALMFC, VISpFC, direction, regulation, difference])

    df_DiffGenes = pd.DataFrame(rows, columns=['Class','Comparison', "Gene", "brain_region", "Conditions", "CCA", 'RCCA', 'RTSS', "avg_logFC", "ALM logFC", "VISp logFC", "Direction", "Regulation (A vs B)", "% (ALM/VISp) logFC"])
    df_DiffGenes = df_DiffGenes.drop_duplicates(subset= ['Class','Comparison', "Gene", 'brain_region'], keep='first')
    df_DiffGenes
    
    df_unconserved = df_unconserved.drop_duplicates(subset= ['Class','Comparison', "Gene", 'brain_region'], keep='first')


    return GO, cellclass, brain_region, df_DiffGenes, df_unconserved

### Subclass ID (SCID) Collapser
requires df to first be passed through DE Collapser - takes a collapsed df (output from DE Collapser) and determines which genes define subclasses across brain regions

In [15]:
def SCID(prepped_df):
    
  #prep:
    prepped_df['UpReg'], prepped_df['DownReg'], prepped_df['First'], prepped_df['Second'] = [0,0,0,0]
               

#           label columns for first and second subclass in DE test to determine up or down regulation
    prepped_df[['First','Second']] = prepped_df['Comparison'].str.split("vs", n = 1, expand = True) 

#           remove unecessary white spaces
    prepped_df['First'] = prepped_df['First'].str.strip()
    prepped_df['Second'] = prepped_df['Second'].str.strip()

#           using idea that FC>0 means first subclass upreg and second subclass down and vice versa, label reg columns
    prepped_df.loc[prepped_df['avg_logFC'] > 0, 'UpReg'] = prepped_df['First']
    prepped_df.loc[prepped_df['avg_logFC'] > 0, 'DownReg'] = prepped_df['Second']

    prepped_df.loc[prepped_df['avg_logFC'] < 0, 'UpReg'] = prepped_df['Second']
    prepped_df.loc[prepped_df['avg_logFC'] < 0, 'DownReg'] = prepped_df['First']
    
    L2_3IT = ['L2/3 IT vs L5 IT', 'L2/3 IT vs L6 CT', 'L2/3 IT vs L5 PT']
    L5IT = ['L2/3 IT vs L5 IT','L5 IT vs L5 PT','L5 IT vs L6 CT']
    IT = ['L2/3 IT vs L5 PT', 'L2/3 IT vs L6 CT', 'L5 IT vs L5 PT','L5 IT vs L6 CT']
    L5PT = ['L2/3 IT vs L5 PT','L5 IT vs L5 PT','L5 PT vs L6 CT']
    L6CT = ['L2/3 IT vs L6 CT','L5 IT vs L6 CT','L5 PT vs L6 CT']
    VIP = ['VIP vs PV', 'VIP vs SST']
    SST = ['SST vs PV', 'VIP vs SST']
    PV = ['SST vs PV', 'VIP vs PV']

#       dictionary for all possible subclasses 
    SubclassDict = {'L2/3 IT': L2_3IT, 'L5 IT': L5IT, 'IT': IT, 'L5 PT': L5PT, 'L6 CT': L6CT, 'VIP': VIP, 'SST': SST, 'PV': PV}

    for gene in prepped_df["Gene"].unique():
        
        df_comparisons = prepped_df[prepped_df["Gene"] == gene]
        
#       can add filter for min number of DE tests for given gene to be required for a subclass ID or can keep as is if interested in all combos
        #solves issue of ASD/Neurodev vs Neurodev/ASD; compList is list of all comparisons involved in DE of a given gene
        
        compList = df_comparisons['ComparisonList'].unique()[0].split(",")
              
#       lists to add subclasses(subclass)/regulation(reg) groups when using DE collapser before Morpheus
        subclass = []
        Reg = []
        
        print("gene = " + gene)
                              
#       identifies all subclasses present in subsetted dataset and sorts them
        SubclassList = list(set(list(df_comparisons['First'].unique()) + list(df_comparisons['Second'].unique())))

#       efficiency optimizing for groups such as IT; e.g. if glut classes are involved keep IT - otherwise remove  
        if (set(['L2/3 IT','L5 IT']).issubset(set(SubclassList))):
            SubclassList.append('IT')
            
        SubclassList = sorted(SubclassList)
        
        SubclassDEList = []
        
#       focus on only subclasses of subsetted data; sorting subclassList sorts SubclassDEList
        for SC in SubclassList:
            SubclassDEList.append(SubclassDict[SC])
#             print(SubclassDEList)
        
        #loop through all potential subclass lists to check for presence of each subclass
        for i in SubclassDEList:
#             print("i equals: " + str(i))

            if (set(i).issubset(set(compList))):

    #           make subset of DE tests for given subclass for a given gene
                df_comparing = df_comparisons[df_comparisons['Comparison'].isin(i)].copy()

    #                 check for conserved direction on group subclasses (e.g. IT)
                if i == IT:
                    df_comparing.loc[(df_comparing['UpReg'] == 'L2/3 IT') | (df_comparing['UpReg'] == 'L5 IT'),'UpReg'] = 'IT'
                    df_comparing.loc[(df_comparing['DownReg'] == 'L2/3 IT') | (df_comparing['DownReg'] == 'L5 IT'), 'DownReg'] = 'IT'

            #check for conserved direction on individual subclasses; does this by checking that only one unique subclass is down or up regulated
                for SC in SubclassList:

#                     print(SC)

    #                       set(list(df_comparing['Down/UpReg']) should only have one subclass for the following to work (hence, why edited above for IT)
                    if (set(list(df_comparing['DownReg'])).issubset(set([SC]))):

                        print("SC downreg equals: " + SC)

                        subclass.append(df_comparing['DownReg'].unique()[0])
                        Reg.append('DownReg')

                    elif (set(list(df_comparing['UpReg'])).issubset(set([SC]))):

                        print("SC upreg equals: " + SC)

                        subclass.append(df_comparing['UpReg'].unique()[0])
                        Reg.append('UpReg')

#       reg/subclass - xSC ySC / up down are paired
       
        prepped_df.loc[prepped_df['Gene'] == gene, 'Subclass'] = ';'.join(subclass)
        prepped_df.loc[prepped_df['Gene'] == gene, 'Direction of Regulation'] = '/'.join(Reg)

    return prepped_df

### CortexSCID (for use only when interested in cortex-specific SCID)
Develops SCID to identify genes that define subclasses in one cortex by incorporating genes that were conserved as well as those exclusively DE in one cortex - addresses issue where failing to consider conserved genes when using SCID overlooks potential subclass labels when using SCID in cortex. 


Also removes genes where SCID labels are already apparent in conserved SCID (i.e. SCID conserved found gene X as upregulated in IT cells but SCID cortex flagged gene X as upregulated in IT and downregulated L5 PT; CortexSCID keeps gene X downregulated for L5 PT)

conserved = SCID output for conserved data

Cortex = SCID output for all ALM or VISp data

In [16]:
def CortexSCID(conserved, Cortex):
    
#   finds genes that are present in conserved and ALM/VISP specific
    overlapGenes = Cortex[['Gene']].drop_duplicates().merge(conserved[['Gene']].drop_duplicates(), how = 'inner' , indicator=False)
    
#   finds genes that are present in conserved and ALM/VISP specific with same ComparisonList and Subclass 
    overlapGenesSC = Cortex[['Gene','ComparisonList','Subclass']].drop_duplicates().merge(conserved[['Gene','ComparisonList','Subclass']].drop_duplicates(), how = 'inner' , indicator=False)

    # special genes
    UniqueSCID = set(overlapGenes.Gene).symmetric_difference(set(overlapGenesSC.Gene))
    
    print(overlapGenesSC)

    # remove genes in ALM/VISp data that have are conserved and have same SCID in cortex and conserved
    Cortex_unique = Cortex.loc[~Cortex['Gene'].isin(overlapGenesSC['Gene'])]

    # deals with genes that are conserved but have a different SCID in cortex and conserved
    UniqueSCID

    for gene in UniqueSCID:
        print(gene)
        
        cortexSC = Cortex_unique.loc[Cortex_unique['Gene'] == gene]['Subclass'].unique()[0].split(";")
        conservedSC = conserved.loc[conserved['Gene'] == gene]['Subclass'].unique()[0].split(";")
        DirectionCortexSC = Cortex_unique.loc[Cortex_unique['Gene'] == gene]['Direction of Regulation'].unique()[0].split("/")

        print("cortexSC = " + str(cortexSC))
        print(DirectionCortexSC)
        print('conservedSC = ' + str(conservedSC))

        if {''}.issubset(conservedSC):
            conservedSC.remove('')

        if {''}.issubset(cortexSC):
            cortexSC.remove('')

#       KeepSCID always pulls out genes with extra SCID labels that aren't in conserved SCID
        KeepSCID = set(cortexSC).symmetric_difference(set(conservedSC))
        KeepSCID = sorted(list(KeepSCID))

    #     KeepSCID = ('/').join((KeepSCID))
        print('Keep ' + str(KeepSCID))

        DirectionList = []
        
#       appends directions with corresponding index of subclass; e.g. linked lists cortexSC = ['PV', 'VIP'] DirectionCortexSC = ['UpReg', 'DownReg']; assume we remove PV since it is flagged in conserved and cortex; thus want to keep ['VIP'] and ['Downreg']   
        for SC in KeepSCID:
            DirectionList.append(DirectionCortexSC[cortexSC.index(SC)])

        print("Direction List = " + str(DirectionList))

        Cortex_unique.loc[Cortex_unique['Gene'] == gene, 'Subclass'] = ('/').join(KeepSCID)
        Cortex_unique.loc[Cortex_unique['Gene'] == gene, 'Direction of Regulation'] = ('/').join(DirectionList)
        
        if(len(conservedSC)>=1):
            Cortex_unique.loc[Cortex_unique['Gene'] == gene, 'Asterisk'] = '*'

    return Cortex_unique

### Signer
pass a number - returns up or down regulation

In [17]:
def signer(number):
    if number > 0:
        return "up-regulation"
    elif number <0:
        return "down-regulation"
    else:
        return "reg 0 or not calculated"

### MorphPrep 

Produces a tuple with 4 df_files (prepped_df, Matrix, Meta_genes, Meta_comparison) that upon saving as CSVs can be directly applied to Broad Institute app Morpheus

relies on all above functions. Option to use DECollapser - if yes, SCID/cortexSCID is also an option

df_clinPrep = df generated from ClinPrep

df_combo = legacy combined dataset

df_ClinList = manually made condition dictionary

In [18]:
def MorphPrep(df_clinPrep, df_combo, df_ClinList):
    
    # clinical or not clinical
    while True: 
        clinical = input("Interested in clinical analysis?(y/n)): ")
        if clinical == 'y':
            df2 = ClinLabel(df_clinPrep, df_combo,df_ClinList)
            break
        elif clinical == 'n':
            df2 = df_combo
            break
        else:
            print("Please enter valid input")
    
#   subset alone or use collapser+subset    
    while True: 
        collapse = input("Would you like to collapse dataset?(y/n) (if dataset is already collapsed, please type 'n'): ")
        
        if collapse == 'y':
            collapser = DECollapser(df2)
            df3 = collapser[3]
            
            cellclass = collapser[1]
            
#           prepare df for comparing for brain-region specific to conserved  
            dfSCIDConserved = collapser[3].copy()
            dfSCIDConserved = dfSCIDConserved.loc[(dfSCIDConserved['Direction'] == 'same')]
            
            D = {'same':'same', 'diff':'different', 'specific': 'specific to subclass'}

            while True: 
                direction = input("Select subclasses with avglogFC in the same, different (diff), or specific to brain region direction (specific): ")
                if (direction == 'diff'):
                    df3 = collapser[4]
                    break
                
                elif (direction == 'same' or direction == 'specific'):
                    
                    if (direction == 'same'):
                        df3 = df3.loc[(df3['Direction'] == D[direction])]
                    
                    elif direction == 'specific':
                        while True:
                            region = input("ALM or VISp: ")
                            
#                          offer opportunity if planning to use SCID so to not lose genes that may have not been exclusively DE but still contribute to subclass ID   
                            SCIDplan = input("Are you planning to ID subclass specific genes on your collapsed brain region specific dataset?(y/n) ")
                            
                            if (region == 'ALM' and SCIDplan == 'y') or (region == 'VISp' and SCIDplan == 'y'):
                                df3 = df3.loc[(df3['brain_region'] == region) | (df3['brain_region'] == 'ALM and VISp')]
                                
#                               to make downstream avg_logFC values consistent with region selected we change average to brain region log_FC; checked splice warning - no error  
                                df3['avg_logFC'] = df3[f'{region} logFC']
                                break
                            
                            elif (region == 'ALM' and SCIDplan == 'n') or (region == 'VISp' and SCIDplan == 'n'):
                                df3 = df3.loc[(df3['brain_region'] == region)]
                                break
    
                            else:
                                print("Please enter valid input")
                    break
                else:
                    print("Please enter valid input")
            break
        
        elif collapse == 'n':
            df3 = subsetter(df2)[3]
            break
        else:
            print("Please enter valid input")
    
    # generate matrix for values:   
    prepped_df = df3.copy()
    
    # make meta_genes sorted as desired ("order")
    prepped_df['Frequency'] = prepped_df.groupby('Gene')['Gene'].transform('count')
    prepped_df['ComparisonList'] = 'NAN'
    

    for gene in prepped_df["Gene"].unique():
        df_comparisons = prepped_df[prepped_df["Gene"] == gene]

        #solves issue of ASD/Neurodev vs Neurodev/ASD; compList is list of all comparisons involved in DE of a given gene
        compList = sorted(df_comparisons['Comparison'].tolist())
        my_string = ','.join(compList)
        prepped_df.loc[prepped_df['Gene'] == gene, "ComparisonList"] = my_string
        
        
    #   repeat for dfSCIDConserved
    if (collapse == 'y'):
        if (direction == 'specific'):
            if SCIDplan == 'y':

                dfSCIDConserved['Frequency'] = dfSCIDConserved.groupby('Gene')['Gene'].transform('count')
                dfSCIDConserved['ComparisonList'] = 'NAN'

                for gene in dfSCIDConserved["Gene"].unique():
                    df_comparisons = dfSCIDConserved[dfSCIDConserved["Gene"] == gene]

                    #solves issue of ASD/Neurodev vs Neurodev/ASD; compList is list of all comparisons involved in DE of a given gene
                    compList = sorted(df_comparisons['Comparison'].tolist())
                    my_string = ','.join(compList)
                    dfSCIDConserved.loc[dfSCIDConserved['Gene'] == gene, "ComparisonList"] = my_string

#   SCID or not SCID
    if (collapse == 'y'):
        if (direction != 'diff'):
            while True: 
                    ID = input("Would you like to ID subclass specific genes on your collapsed dataset?(y/n) ")
                    if ID == 'y':

                        prepped_df = SCID(prepped_df)                                                

                    # if looking for motifs specific to brain regions, must take out class-relevant genes present across both brain regions and only keep those in which there is additional region-specific class info  
                        if direction == 'specific':

                            if SCIDplan == 'y':

                                dfSCIDConserved = SCID(dfSCIDConserved)
                                prepped_df = CortexSCID(dfSCIDConserved,prepped_df)                            

                        break
                    elif ID == 'n':
                        break
                    else:
                        print("Please enter valid input")
                                 
    prepped_df['Subclass'] = prepped_df['Subclass'].str.replace(";","/")

    prepped_df['ComparisonList'] = prepped_df['ComparisonList'].str.replace(',','/')
    
#   corrects diamond key - assures first subclass listed upregulates gene and second subclass listed downregulates gene 
    if (collapse == 'y'):
        if (direction != 'diff'):
            if (cellclass != 'all'):
                prepped_df.loc[prepped_df['Direction of Regulation'] == 'DownReg/UpReg', 'Subclass'] = prepped_df.loc[prepped_df['Direction of Regulation'] == 'DownReg/UpReg']['Subclass'].apply(lambda x: ((',').join(x.split('/')[::-1])).replace(',','/'))
                prepped_df.loc[prepped_df['Direction of Regulation'] == 'DownReg/UpReg', 'Direction of Regulation'] = prepped_df.loc[prepped_df['Direction of Regulation'] == 'DownReg/UpReg']['Direction of Regulation'].apply(lambda x: ((',').join(x.split('/')[::-1])).replace(',','/'))
    
    df_matrix = prepped_df[['Gene','Comparison','avg_logFC']]
    pivot = df_matrix.pivot('Gene', 'Comparison','avg_logFC')
    Matrix = pivot
    
    # generate matrix for gene annotations:    
    if clinical == 'y':
        
#       pre-req for ID  
        if (collapse == 'y'):
        
            if (direction != 'diff'):
            
                if ID == 'y':
                    prepped_df.sort_values(['Conditions', 'Subclass', 'Frequency','ComparisonList', 'Direction of Regulation','Gene'], inplace=True, ascending=[True,False,False,True, False, True])
                    prepped_df['Order'] = range(1, 1+len(prepped_df))

                    df_genes = prepped_df[['Gene','ComparisonList','CCA', 'RCCA','RTSS','Frequency','Subclass','Direction of Regulation','Conditions', 'Order']]
        
        else:
            prepped_df.sort_values(['Conditions', 'Frequency','ComparisonList','Gene'], inplace=True, ascending=[True,False,True,True])
            prepped_df['Order'] = range(1, 1+len(prepped_df))
            df_genes = prepped_df[['Gene','ComparisonList', 'Frequency','CCA', 'RCCA','RTSS','Conditions', 'Order']]
        
#       checked splice warning, not an issue
        df_genes['Conditions'] = df_genes['Conditions'].str.replace(',','/')
    
    else:
         if (collapse == 'y'):
            if (direction != 'diff'):
                if ID == 'y':
                    prepped_df.sort_values(['Subclass','ComparisonList','Frequency', 'Direction of Regulation','Gene'], inplace=True, ascending=[True,False,False, False, True])
                    prepped_df['Order'] = range(1, 1+len(prepped_df))
                    df_genes = prepped_df[['Gene','ComparisonList','CCA', 'RCCA','RTSS','Frequency','Subclass','Direction of Regulation','Order']]
        
         else:
            prepped_df['Order'] = range(1, 1+len(prepped_df))
            prepped_df.sort_values(['Frequency','ComparisonList','Gene'], inplace=True, ascending=[False,True,True])
            df_genes = prepped_df[['Gene','ComparisonList', 'Frequency','CCA', 'RCCA','RTSS', 'Order']]
  
    if (collapse == 'y'):
        if (direction == 'diff'):
            df_genes = prepped_df[['Gene','ComparisonList', 'Frequency','CCA', 'RCCA','RTSS']]      
        
#   in order to prepare for Morpheus, we drop redundant gene rows; checked splice warning, not an issue   
    df_genes.drop_duplicates(subset=['Gene'], keep="first", inplace = True)

    # generate matrix for comparison annotations: (change to prepped_df)
    df_comparison = prepped_df[['Comparison','brain_region','Class', 'Subclass']]
    Meta_comparison = df_comparison
    
    Meta_genes = df_genes        

    return prepped_df, Matrix, Meta_genes, Meta_comparison

### Morpheus creator testing terminal


In [23]:
import numpy as np
import pandas as pd

# to run, change file paths to locations on your computer. For questions concerning each file, review readme

df_clin= pd.read_csv(r'C:\SFARI.csv')

df_clin = clinExploder(df_clin)

df_DEGenes = pd.read_csv(r'C:\DE Gene List.csv')

df_clinPrep = ClinPrep(df_clin,df_DEGenes)

df_combo = pd.read_csv(r'C:\Combined_filtered_GO.csv')

df_ClinList = pd.read_csv(r'C:\Narrowed Condition List.csv')

MorphFiles = MorphPrep(df_clinPrep, df_combo, df_ClinList)

In [ ]:
# view ouput df from applying tools
analyzed_df = MorphFiles[0]

In [22]:
# Morpehus saver testing terminal - save files for uploading to Morpheus and visualizing

pivot = MorphFiles[1]

df_genes = MorphFiles[2]

df_comparison = MorphFiles[3]

pivot.to_csv(r'C:\Matrix.csv')

df_genes.to_csv(r'C:\Meta_genes.csv', index = False)

df_comparison.to_csv(r'C:\Meta_comparison.csv', index = False)